In [2]:
# repo url(https://github.com/xwang71785/buffett.git)
# 只针对中国大陆股票市场
# 仅限于概念验证，勿用于实际投资中
# QQ：2908683294
import numpy as np
import pandas as pd


In [3]:
server = "http://quotes.money.163.com/"
path = "service/"
stock = "600585"    # 根据需求变更stock中的股票代码
type = ".html?type=year"

In [4]:
# 网易也提供了比较完整的财务指标数据
table4 = "zycwzb_"
part5 = "&part=ylnl"
part6 = "&part=chnl"
part7 = "&part=cznl"
part8 = "&part=yynl"

target4 = server + path + table4 + stock + type
target5 = server + path + table4 + stock + type + part5
target6 = server + path + table4 + stock + type + part6
target7 = server + path + table4 + stock + type + part7
target8 = server + path + table4 + stock + type + part8

print(target4, '\n', target5, '\n', target6, '\n', target7, '\n', target8)

http://quotes.money.163.com/service/zycwzb_600585.html?type=year 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=ylnl 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=chnl 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=cznl 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=yynl


#### 估值2
股利贴现模型 Dividend Discounted Model

In [ ]:
# 用股息平均增长率G来计算
# A股的分红方案极不稳定，不太适合标准的DDM： P0 = D1 /（K - G）且 K>G, G是分红的增长率

url1 = "http://quotes.money.163.com/f10/fhpg_002270.html#01d05"
with urllib.request.urlopen(url1) as i:
    html = i.read()

In [ ]:
data1 = pd.read_html(html)[3]    # html中第4个表才是分红数据
col = np.array(data1.columns.levels[0])    # 读人DataFrame中的数据是双层列名的
data1.columns = ['公告日期','分红年度','送股','转增','派息','股权登记日','除权除息日','红股上市日']    # 对DataFrame重命名
if col[2] == '分红方案（每10股）':    # 判断分红方案是否按10股派息
    data1['派息'] = data1['派息'] / 10
data1

In [ ]:
dividend = pd.DataFrame()
dividend['year'] = data1['分红年度']
dividend['div'] = data1['派息']
dividend['growth'] = (data1['派息'] - data1['派息'].shift(-1)) / data1['派息'].shift(-1)

In [ ]:
K = 0.12    # K贴现率
dividend0 = dividend.iloc[0,1]
growth0 = dividend.iloc[0,2]
estimated_price0 = 0
if growth0 < K:
    estimated_price0 = dividend0 * (1 + growth0) / (K - growth0)
print(estimated_price0)    # 如果估值为0代表分红增长率不收敛

In [ ]:
# 可持续增长率(Sustainable Growth Rate)
# ROE = Net income / Equity
# Payout Ratio = Dividend / EPS 
# SGR = ROE * (1-Payout Ratio)
# 

In [6]:
# 获取主要财务指标文件
df4 = pd.read_csv(target4, header=0, encoding="gb18030")    # 主要财务指标
df5 = pd.read_csv(target5, header=0, encoding="gb18030")    # 盈利能力
#df6 = pd.read_csv(target6, header=0, encoding="gb18030")    # 偿还能力
#df7 = pd.read_csv(target7, header=0, encoding="gb18030")    # 成长能力
#df8 = pd.read_csv(target8, header=0, encoding="gb18030")    # 营运能力

In [10]:
financial = pd.DataFrame(df4.values.T, columns=df4['报告日期'], index=df4.columns)
financial.drop('报告日期', inplace=True)    # 删除第一行

In [13]:
headers_financial = financial.columns

for head in headers_financial:
    financial[head] = pd.to_numeric(financial[head], errors='coerce').fillna(0) # 不符合转换条件的设为‘Nan’，再用0替换


In [14]:
price_ratio = pd.DataFrame()
price_ratio['EPS'] = financial['基本每股收益(元)']
price_ratio['APS'] = financial['每股净资产(元)']
price_ratio['CFPS'] = financial['每股经营活动产生的现金流量净额(元)']
price_ratio['ROE'] = financial['净资产收益率加权(%)']
price_ratio['growth_rate'] = (financial['基本每股收益(元)'] - financial['基本每股收益(元)'].shift(-1)) / financial['基本每股收益(元)'].shift(-1)

# 基本每股收益(元) = PL.归属母公司所有者的净利润 / BS.实收资本
# 每股净资产(元) = BS.归属母公司股东权益合计 / BS.实收资本
# 每股经营活动产生的现金流量净额(元) = CF.经营活动产生的现金流量净额 / BS.实收资本
# 净资产收益率加权(%) = PL.净利润 / BS.归属母公司股东权益合计
price_ratio.head()

,EPS,APS,CFPS,ROE,growth_rate
2021-12-31,6.28,34.66,6.40,19.23,-0.052790
2020-12-31,6.63,30.54,6.57,23.62,0.045741
2019-12-31,6.34,25.92,7.69,27.03,0.126110
2018-12-31,5.63,21.26,6.80,29.66,0.882943
2017-12-31,2.99,16.87,3.28,19.12,0.857143
2016-12-31,1.61,14.46,2.49,11.59,0.133803
2015-12-31,1.42,13.30,1.87,11.03,-0.314010
2014-12-31,2.07,12.50,3.33,18.47,0.169492
2013-12-31,1.77,10.59,2.87,18.05,0.487395
2012-12-31,1.19,9.23,2.17,13.50,-0.456621


In [13]:
growth = pd.DataFrame(df7.values.T, columns=df7['报告日期'], index=df7.columns)
growth.drop('报告日期', inplace=True)    # 删除第一行
growth.columns = ['revenue','profit','net_assets','total_assets']
growth.head()

,revenue,profit,net_assets,total_assets
2021-12-31,-4.7,-6.06,14.04,14.7
2020-12-31,12.23,5.88,18.2,12.42
2019-12-31,22.3,12.13,22.25,19.55
2018-12-31,70.5,86.48,26.59,22.44
2017-12-31,34.65,83.55,14.53,11.53
2016-12-31,9.72,17.34,8.64,3.53
2015-12-31,-16.1,-34.17,6.18,3.45
2014-12-31,9.95,18.1,18.46,9.84
2013-12-31,20.75,51.83,14.83,6.37
2012-12-31,-5.94,-45.35,9.25,4.19


Expected share price = P/E_ratio * EPS * (1 + EPS_growth_rate)

Expected share price = P/CF_ratio * CFPS * (1 + CFPS_growth_rate)

In [15]:
# 设置最近股价的日期
import datetime as dt 
endday = dt.datetime.today()    # 当天日期
if endday.weekday() == 6:    # 当天是否周日
    end = dt.datetime(endday.year, endday.month, endday.day - 2)
if endday.weekday() == 5:    # 当天是否周六
    end = dt.datetime(endday.year, endday.month, endday.day - 1)    # 把当日设为周五

start = end - dt.timedelta(3,0,0)    # 开始日期往前到推3天
start = start.strftime('%Y%m%d')    # 把日期转换为字符串
end = end.strftime('%Y%m%d')
print(end)

20220805


In [16]:
# 股票代码, 沪市加0，深市加1
if stock[0:1] == '6':
    code = '0' + stock
elif stock[0:1] == '0':
    code = '1' + stock
elif stock[0:1] == '3':
    code = '1' + stock

print(code)

0600585


In [17]:
# 基于网易构造历史成交数据源URL
type = "chddata.html?code="
start = "&start=" + start
end = "&end=" + end
target9 = server + path + type + code + start + end

print(target9)

http://quotes.money.163.com/service/chddata.html?code=0600585&start=20220802&end=20220805


In [20]:
# 获取成交历史原始数据
df9 = pd.read_csv(target9, header=0, encoding="gb18030")
price = df9.loc[0, '收盘价']
print(price)

32.03


In [29]:
price * (1 + price_ratio.iloc[0, 4])

30.339125188536958